In [ ]:
!pip install niapy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from niapy.task import Task
from niapy.problems import Sphere
from niapy.problems import Problem

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/ott-deceptive-opinion.csv",encoding="ISO-8859-1")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 1,600



,deceptive,hotel,polarity,source,text
707,deceptive,james,positive,MTurk,My stay at the James Hotel in Chicago was fant...
884,truthful,homewood,negative,Web,"I called on December 6th, 2010 at 3 p.m. to bo..."
530,deceptive,sheraton,positive,MTurk,I stayed at the Sheraton Chicago Hotel and Tow...
1089,truthful,james,negative,Web,While the James was once considered an elegant...
481,deceptive,sheraton,positive,MTurk,Thanks Sheraton Towers for the invite to enjoy...
375,truthful,palmer,positive,TripAdvisor,Spent three nights at the hotel for a girls we...
1410,deceptive,hardrock,negative,MTurk,I would not reccomend staying at the Hard Rock...
1524,deceptive,intercontinental,negative,MTurk,"I've never written a review before, but I just..."
1396,deceptive,ambassador,negative,MTurk,My husband and I recently went to Chicago for ...
632,deceptive,hardrock,positive,MTurk,I traveled to Chicago recently with my giant G...


In [ ]:
text=df['text']
label = df['deceptive']

In [ ]:
print (text[1])


Triple A rate with upgrade to view room was less than $200 which also included breakfast vouchers. Had a great view of river, lake, Wrigley Bldg. & Tribune Bldg. Most major restaurants, Shopping, Sightseeing attractions within walking distance. Large room with a very comfortable bed. 



In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
yt = le.fit_transform(label)
print(label)
print(yt)

0        truthful
1        truthful
2        truthful
3        truthful
4        truthful
          ...    
1595    deceptive
1596    deceptive
1597    deceptive
1598    deceptive
1599    deceptive
Name: deceptive, Length: 1600, dtype: object
[1 1 1 ... 0 0 0]


In [ ]:
print(df.isnull().sum())

deceptive    0
hotel        0
polarity     0
source       0
text         0
dtype: int64


In [ ]:
#import transformers
from transformers import BertTokenizer
from transformers import DebertaTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base', do_lower_case=True)


Loading BERT tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Print the original sentence.
print(' Original: ', text[0])

 Original:  We stayed for a one night getaway with family on a thursday. Triple AAA rate of 173 was a steal. 7th floor room complete with 44in plasma TV bose stereo, voss and evian water, and gorgeous bathroom(no tub but was fine for us) Concierge was very helpful. You cannot beat this location... Only flaw was breakfast was pricey and service was very very slow(2hours for four kids and four adults on a friday morning) even though there were only two other tables in the restaurant. Food was very good so it was worth the wait. I would return in a heartbeat. A gem in chicago... 



In [ ]:
print('Tokenized: ', tokenizer.tokenize(text[0]))

Tokenized:  ['We', 'Ġstayed', 'Ġfor', 'Ġa', 'Ġone', 'Ġnight', 'Ġget', 'away', 'Ġwith', 'Ġfamily', 'Ġon', 'Ġa', 'Ġth', 'ursday', '.', 'ĠTriple', 'ĠAAA', 'Ġrate', 'Ġof', 'Ġ173', 'Ġwas', 'Ġa', 'Ġsteal', '.', 'Ġ7', 'th', 'Ġfloor', 'Ġroom', 'Ġcomplete', 'Ġwith', 'Ġ44', 'in', 'Ġplasma', 'ĠTV', 'Ġb', 'ose', 'Ġstereo', ',', 'Ġv', 'oss', 'Ġand', 'Ġev', 'ian', 'Ġwater', ',', 'Ġand', 'Ġgorgeous', 'Ġbathroom', '(', 'no', 'Ġtub', 'Ġbut', 'Ġwas', 'Ġfine', 'Ġfor', 'Ġus', ')', 'ĠConc', 'ier', 'ge', 'Ġwas', 'Ġvery', 'Ġhelpful', '.', 'ĠYou', 'Ġcannot', 'Ġbeat', 'Ġthis', 'Ġlocation', '...', 'ĠOnly', 'Ġflaw', 'Ġwas', 'Ġbreakfast', 'Ġwas', 'Ġpricey', 'Ġand', 'Ġservice', 'Ġwas', 'Ġvery', 'Ġvery', 'Ġslow', '(', '2', 'hours', 'Ġfor', 'Ġfour', 'Ġkids', 'Ġand', 'Ġfour', 'Ġadults', 'Ġon', 'Ġa', 'Ġfr', 'iday', 'Ġmorning', ')', 'Ġeven', 'Ġthough', 'Ġthere', 'Ġwere', 'Ġonly', 'Ġtwo', 'Ġother', 'Ġtables', 'Ġin', 'Ġthe', 'Ġrestaurant', '.', 'ĠFood', 'Ġwas', 'Ġvery', 'Ġgood', 'Ġso', 'Ġit', 'Ġwas', 'Ġworth', 'Ġthe', 'Ġ

In [ ]:
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[0])))

Token IDs:  [170, 4711, 13, 10, 65, 363, 120, 4384, 19, 284, 15, 10, 3553, 46806, 4, 8424, 17147, 731, 9, 30011, 21, 10, 8052, 4, 262, 212, 1929, 929, 1498, 19, 3550, 179, 29051, 1012, 741, 3876, 31436, 6, 748, 5434, 8, 7630, 811, 514, 6, 8, 12058, 8080, 1640, 2362, 17465, 53, 21, 2051, 13, 201, 43, 21657, 906, 1899, 21, 182, 7163, 4, 370, 1395, 1451, 42, 2259, 734, 4041, 22892, 21, 7080, 21, 26428, 8, 544, 21, 182, 182, 2635, 1640, 176, 21719, 13, 237, 1159, 8, 237, 3362, 15, 10, 6664, 21746, 662, 43, 190, 600, 89, 58, 129, 80, 97, 9248, 11, 5, 2391, 4, 3652, 21, 182, 205, 98, 24, 21, 966, 5, 2067, 4, 38, 74, 671, 11, 10, 28288, 4, 83, 15538, 11, 1855, 42938, 734, 1437, 50118]


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in text:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent[:512],                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)


In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', text[1])
print('Token IDs:', input_ids[1])

Original:  Triple A rate with upgrade to view room was less than $200 which also included breakfast vouchers. Had a great view of river, lake, Wrigley Bldg. & Tribune Bldg. Most major restaurants, Shopping, Sightseeing attractions within walking distance. Large room with a very comfortable bed. 

Token IDs: [1, 35587, 8293, 83, 731, 19, 6662, 7, 1217, 929, 21, 540, 87, 68, 2619, 61, 67, 1165, 7080, 25137, 4, 7301, 10, 372, 1217, 9, 4908, 6, 8037, 6, 305, 7638, 607, 163, 4779, 571, 4, 359, 7990, 163, 4779, 571, 4, 1993, 538, 4329, 6, 21261, 6, 31833, 33471, 16530, 624, 3051, 4472, 4, 13769, 929, 19, 10, 182, 3473, 3267, 4, 1437, 50118, 2]


In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  143


In [ ]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 512 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length ...
MAX_LEN = 256

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 256 values...

Padding token: "[PAD]", ID: 0

Done.


In [ ]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:

    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for
# training

from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, yt,
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, yt,
                                             random_state=2018, test_size=0.1)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
'''# Find the minimum size
min_size = min(train_inputs.size(0), train_masks.size(0), train_labels.size(0))

# Trim the tensors to the minimum size
train_inputs = train_inputs[:min_size]
train_masks = train_masks[:min_size]
train_labels = train_labels[:min_size]

# Create the DataLoader for the training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


min_size = min(validation_inputs.size(0), validation_masks.size(0), validation_labels.size(0))

# Trim the tensors to the minimum size
validation_inputs = validation_inputs[:min_size]
validation_masks = validation_masks[:min_size]
validation_labels = validation_labels[:min_size]

# Create the DataLoader for the training set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)'''


'# Find the minimum size\nmin_size = min(train_inputs.size(0), train_masks.size(0), train_labels.size(0))\n\n# Trim the tensors to the minimum size\ntrain_inputs = train_inputs[:min_size]\ntrain_masks = train_masks[:min_size]\ntrain_labels = train_labels[:min_size]\n\n# Create the DataLoader for the training set\ntrain_data = TensorDataset(train_inputs, train_masks, train_labels)\ntrain_sampler = RandomSampler(train_data)\ntrain_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)\n\n\nmin_size = min(validation_inputs.size(0), validation_masks.size(0), validation_labels.size(0))\n\n# Trim the tensors to the minimum size\nvalidation_inputs = validation_inputs[:min_size]\nvalidation_masks = validation_masks[:min_size]\nvalidation_labels = validation_labels[:min_size]\n\n# Create the DataLoader for the training set\nvalidation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)\nvalidation_sampler = RandomSampler(validation_data)\nvali

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import get_linear_schedule_with_warmup


In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup, DebertaForSequenceClassification
from niapy.task import Task
from niapy.problems import Problem
from niapy.algorithms.basic import GreyWolfOptimizer,FireflyAlgorithm, HarrisHawksOptimization, MonarchButterflyOptimization, LionOptimizationAlgorithm, BatAlgorithm, BeesAlgorithm, BacterialForagingOptimization, CoralReefsOptimization,  CuckooSearch, CatSwarmOptimization, ForestOptimizationAlgorithm, GlowwormSwarmOptimization, MothFlameOptimizer, ParticleSwarmOptimization

# Define GWO problem for hyperparameter optimization
class HyperparameterOptimization(Problem):
    def __init__(self):
        # Dimension 2: learning rate and optimizer type
        super().__init__(dimension=2, lower=[1e-6, 0], upper=[1e-3, 6], type='float')

    def _evaluate(self, x):
        learning_rate = x[0]
        optimizer_choice = int(round(x[1]))

        # Choose optimizer
        if optimizer_choice == 0:
            optimizer_class = torch.optim.AdamW
        elif optimizer_choice == 1:
            optimizer_class = torch.optim.SGD
        elif optimizer_choice == 2:
            optimizer_class = torch.optim.Adam
        elif optimizer_choice == 3:
            optimizer_class = lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9, nesterov=True)
        elif optimizer_choice == 4:
            optimizer_class = torch.optim.NAdam
        elif optimizer_choice == 5:
            optimizer_class = torch.optim.RMSprop
        elif optimizer_choice == 6:
            optimizer_class = torch.optim.Adagrad



        # Load the DeBERTa model
        model = DebertaForSequenceClassification.from_pretrained(
            "microsoft/deberta-base",
            num_labels=2,
            output_attentions=False,
            output_hidden_states=False,
        )
        model.cuda()

        # Optimizer and scheduler
        optimizer = optimizer_class(model.parameters(), lr=learning_rate)
        total_steps = len(train_dataloader) * 1  # 1 epoch for quick evaluation
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

        # Training
        model.train()
        total_loss = 0
        for batch in train_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / len(train_dataloader)

        return avg_train_loss

# Run GWO for hyperparameter optimization
task = Task(problem=HyperparameterOptimization(), max_evals=10)
algo = MonarchButterflyOptimization()
best = algo.run(task)

best_learning_rate = best[0][0]
best_optimizer_choice = int(round(best[0][1]))
optimizer_types = ['AdamW', 'SGD', 'Adam', 'NAG', 'Nadam', 'RMSprop', 'Adagrad']
best_optimizer = optimizer_types[best_optimizer_choice]

print(f"Best learning rate: {best_learning_rate}, Best optimizer: {best_optimizer}")


'import torch\nfrom torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset\nfrom transformers import AdamW, get_linear_schedule_with_warmup, DebertaForSequenceClassification\nfrom niapy.task import Task\nfrom niapy.problems import Problem\nfrom niapy.algorithms.basic import GreyWolfOptimizer,FireflyAlgorithm, HarrisHawksOptimization, MonarchButterflyOptimization, LionOptimizationAlgorithm, BatAlgorithm, BeesAlgorithm, BacterialForagingOptimization, CoralReefsOptimization,  CuckooSearch, CatSwarmOptimization, ForestOptimizationAlgorithm, GlowwormSwarmOptimization, MothFlameOptimizer, ParticleSwarmOptimization\n\n# Define GWO problem for hyperparameter optimization\nclass HyperparameterOptimization(Problem):\n    def __init__(self):\n        # Dimension 2: learning rate and optimizer type\n        super().__init__(dimension=2, lower=[1e-6, 0], upper=[1e-3, 6], type=\'float\')\n\n    def _evaluate(self, x):\n        learning_rate = x[0]\n        optimizer_choi

In [ ]:
best_learning_rate = best[0][0]
best_optimizer_choice = int(round(best[0][1]))
optimizer_types = ['AdamW', 'SGD', 'Adam', 'NAG', 'Nadam', 'RMSprop', 'Adagrad']
best_optimizer = optimizer_types[best_optimizer_choice]

print(f"Best learning rate: {best_learning_rate}, Best optimizer: {best_optimizer}")

'best_learning_rate = best[0][0]\nbest_optimizer_choice = int(round(best[0][1]))\noptimizer_types = [\'AdamW\', \'SGD\', \'Adam\', \'NAG\', \'Nadam\', \'RMSprop\', \'Adagrad\']\nbest_optimizer = optimizer_types[best_optimizer_choice]\n\nprint(f"Best learning rate: {best_learning_rate}, Best optimizer: {best_optimizer}")'

In [ ]:
'''import torch
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers import DebertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from niapy.task import Task
from niapy.problems import Problem
from niapy.algorithms.basic import GreyWolfOptimizer,MonarchButterflyOptimization

# Assuming you have already defined train_inputs, train_masks, train_labels,
# validation_inputs, validation_masks, validation_labels, test_inputs, test_masks, test_labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define GWO problem for hyperparameter optimization
class HyperparameterOptimization(Problem):
    def __init__(self):
        super().__init__(dimension=2, lower=[1e-6, 0], upper=[1e-3, 6], type='float')

    def _evaluate(self, x):
        learning_rate = x[0]
        optimizer_choice = int(round(x[1]))

        # Choose optimizer
        if optimizer_choice == 0:
            optimizer_class = torch.optim.AdamW
        elif optimizer_choice == 1:
            optimizer_class = torch.optim.SGD
        elif optimizer_choice == 2:
            optimizer_class = torch.optim.Adam
        elif optimizer_choice == 3:
            optimizer_class = lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9, nesterov=True)
        elif optimizer_choice == 4:
            optimizer_class = torch.optim.Nadam
        elif optimizer_choice == 5:
            optimizer_class = torch.optim.RMSprop
        elif optimizer_choice == 6:
            optimizer_class = torch.optim.Adagrad
        else:
            return float('inf')  # Invalid optimizer choice

        batch_size = 16  # Fixed batch size for simplicity

        # Define DataLoader
        train_data = TensorDataset(train_inputs, train_masks, train_labels)
        train_sampler = SequentialSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

        validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
        validation_sampler = SequentialSampler(validation_data)
        validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

        # Load the DeBERTa model
        model = DebertaForSequenceClassification.from_pretrained(
            "microsoft/deberta-base",
            num_labels=2,
            output_attentions=False,
            output_hidden_states=False,
        )
        model.to(device)

        # Optimizer and scheduler
        optimizer = optimizer_class(model.parameters(), lr=learning_rate)
        total_steps = len(train_dataloader) * 1  # 1 epoch for quick evaluation
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

        # Training
        model.train()
        total_loss = 0
        for batch in train_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / len(train_dataloader)

        return avg_train_loss  # Only return the loss for the optimizer

# Define a function for testing with the best hyperparameters
def test_model(model, test_dataloader):
    model.eval()
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)

        predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())

    report = classification_report(true_labels, predictions)
    return report

# Run GWO for hyperparameter optimization
task = Task(problem=HyperparameterOptimization(), max_evals=1)
algo = MonarchButterflyOptimization()
best_result = algo.run(task)

print(f"Best learning rate: {best_result[0][0]}, Best optimizer choice: {int(round(best_result[0][1]))}")

# Assuming you have test_inputs, test_masks, test_labels prepared similarly to train and validation sets
batch_size = 16  # Ensure consistent batch size
test_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Test the best model
# Recreate the best model and optimizer
best_learning_rate = best_result[0][0]
best_optimizer_choice = int(round(best_result[0][1]))
if best_optimizer_choice == 0:
    best_optimizer_class = torch.optim.AdamW
elif best_optimizer_choice == 1:
    best_optimizer_class = torch.optim.SGD
elif best_optimizer_choice == 2:
    best_optimizer_class = torch.optim.Adam
elif best_optimizer_choice == 3:
    best_optimizer_class = lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9, nesterov=True)
elif best_optimizer_choice == 4:
    best_optimizer_class = torch.optim.Nadam
elif best_optimizer_choice == 5:
    best_optimizer_class = torch.optim.RMSprop
elif best_optimizer_choice == 6:
    best_optimizer_class = torch.optim.Adagrad

best_model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)
best_model.to(device)
best_optimizer = best_optimizer_class(best_model.parameters(), lr=best_learning_rate)

# Test the best model
test_report = test_model(best_model, validation_dataloader)
print("Test Report:")
'''


'import torch\nfrom torch.utils.data import DataLoader, SequentialSampler, TensorDataset\nfrom transformers import DebertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup\nfrom sklearn.metrics import classification_report\nfrom niapy.task import Task\nfrom niapy.problems import Problem\nfrom niapy.algorithms.basic import GreyWolfOptimizer,MonarchButterflyOptimization\n\n# Assuming you have already defined train_inputs, train_masks, train_labels,\n# validation_inputs, validation_masks, validation_labels, test_inputs, test_masks, test_labels\n\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n# Define GWO problem for hyperparameter optimization\nclass HyperparameterOptimization(Problem):\n    def __init__(self):\n        super().__init__(dimension=2, lower=[1e-6, 0], upper=[1e-3, 6], type=\'float\')\n\n    def _evaluate(self, x):\n        learning_rate = x[0]\n        optimizer_choice = int(round(x[1]))\n\n        # Choose optimizer\n        if op

In [ ]:
'''import torch
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers import DebertaForSequenceClassification, DebertaConfig, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from niapy.task import Task
from niapy.problems import Problem
from niapy.algorithms.basic import MonarchButterflyOptimization

# Assuming you have already defined train_inputs, train_masks, train_labels,
# validation_inputs, validation_masks, validation_labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define GWO problem for hyperparameter optimization
class HyperparameterOptimization(Problem):
    def __init__(self):
        super().__init__(dimension=8, lower=[128, 1, 1, 512, 0.1, 1e-6, 1, 0.9], upper=[1024, 24, 24, 4096, 0.5, 1e-2, 1000, 0.999], type='float')

    def _evaluate(self, x):
        params = {
            "hidden_size": int(round(x[0])),
            "num_hidden_layers": int(round(x[1])),
            "num_attention_heads": int(round(x[2])),
            "intermediate_size": int(round(x[3])),
            "hidden_dropout_prob": x[4],
            "learning_rate": x[5],
            "num_train_epochs": int(round(x[6])),
            "adam_beta2": x[7],
        }

        # Ensure hidden_size is divisible by num_attention_heads
        if params["hidden_size"] % params["num_attention_heads"] != 0:
            return float('inf')  # Return a large value to avoid these invalid configurations

        batch_size = 16  # Fixed batch size for simplicity

        # Define DataLoader
        train_data = TensorDataset(train_inputs, train_masks, train_labels)
        train_sampler = SequentialSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

        validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
        validation_sampler = SequentialSampler(validation_data)
        validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

        # Load the DeBERTa model configuration
        model_config = DebertaConfig(
            hidden_size=params["hidden_size"],
            num_hidden_layers=params["num_hidden_layers"],
            num_attention_heads=params["num_attention_heads"],
            intermediate_size=params["intermediate_size"],
            hidden_dropout_prob=params["hidden_dropout_prob"],
        )

        # Load the DeBERTa model
        model = DebertaForSequenceClassification.from_pretrained(
            "microsoft/deberta-base",
            config=model_config,
            ignore_mismatched_sizes=True  # Ignore size mismatches
        )
        model.to(device)

        # Optimizer and scheduler
        optimizer = torch.optim.AdamW(model.parameters(), lr=params["learning_rate"], betas=(0.9, params["adam_beta2"]))
        total_steps = len(train_dataloader) * params["num_train_epochs"]
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)

        # Training
        model.train()
        total_loss = 0
        for epoch in range(params["num_train_epochs"]):
            for batch in train_dataloader:
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)
                model.zero_grad()
                outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                loss = outputs.loss
                total_loss += loss.item()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()

        avg_train_loss = total_loss / (len(train_dataloader) * params["num_train_epochs"])

        # Evaluation on validation set
        model.eval()
        predictions = []
        true_labels = []

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            with torch.no_grad():
                outputs = model(b_input_ids, attention_mask=b_input_mask)

            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(b_labels.cpu().numpy())

        f1 = f1_score(true_labels, predictions)
        return -f1  # Minimize negative F1 score

# Run GWO for hyperparameter optimization
task = Task(problem=HyperparameterOptimization(), max_evals=10)
algo = MonarchButterflyOptimization()
best_result = algo.run(task)

# Print the best hyperparameters found
best_params_dict = {
    "hidden_size": int(round(best_result[0][0])),
    "num_hidden_layers": int(round(best_result[0][1])),
    "num_attention_heads": int(round(best_result[0][2])),
    "intermediate_size": int(round(best_result[0][3])),
    "hidden_dropout_prob": best_result[0][4],
    "learning_rate": best_result[0][5],
    "num_train_epochs": int(round(best_result[0][6])),
    "adam_beta2": best_result[0][7],
}

print(f"Best hyperparameters: {best_params_dict}")'''


'import torch\nfrom torch.utils.data import DataLoader, SequentialSampler, TensorDataset\nfrom transformers import DebertaForSequenceClassification, DebertaConfig, get_linear_schedule_with_warmup\nfrom sklearn.metrics import f1_score\nfrom niapy.task import Task\nfrom niapy.problems import Problem\nfrom niapy.algorithms.basic import MonarchButterflyOptimization\n\n# Assuming you have already defined train_inputs, train_masks, train_labels,\n# validation_inputs, validation_masks, validation_labels\n\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n# Define GWO problem for hyperparameter optimization\nclass HyperparameterOptimization(Problem):\n    def __init__(self):\n        super().__init__(dimension=8, lower=[128, 1, 1, 512, 0.1, 1e-6, 1, 0.9], upper=[1024, 24, 24, 4096, 0.5, 1e-2, 1000, 0.999], type=\'float\')\n\n    def _evaluate(self, x):\n        params = {\n            "hidden_size": int(round(x[0])),\n            "num_hidden_layers": int(round(x[1])),\n

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import DebertaForSequenceClassification, DebertaTokenizer

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)


# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

In [ ]:
!nvidia-smi

Sun Jun 30 14:22:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0              30W /  70W |    693MiB / 15360MiB |     17%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))



The BERT model has 200 different named parameters.

==== Embedding Layer ====

deberta.embeddings.word_embeddings.weight               (50265, 768)
deberta.embeddings.LayerNorm.weight                           (768,)
deberta.embeddings.LayerNorm.bias                             (768,)
deberta.encoder.layer.0.attention.self.q_bias                 (768,)
deberta.encoder.layer.0.attention.self.v_bias                 (768,)

==== First Transformer ====

deberta.encoder.layer.0.attention.self.in_proj.weight    (2304, 768)
deberta.encoder.layer.0.attention.self.pos_proj.weight    (768, 768)
deberta.encoder.layer.0.attention.self.pos_q_proj.weight   (768, 768)
deberta.encoder.layer.0.attention.self.pos_q_proj.bias        (768,)
deberta.encoder.layer.0.attention.output.dense.weight     (768, 768)
deberta.encoder.layer.0.attention.output.dense.bias           (768,)
deberta.encoder.layer.0.attention.output.LayerNorm.weight       (768,)
deberta.encoder.layer.0.attention.output.LayerNorm.bias     

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=8.284448858002006e-05)

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:

import torch.nn as nn
import random
import numpy as np
import torch
import time
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss and accuracy after each epoch for plotting
train_loss_values = []
train_accuracy_values = []
validation_loss_values = []
validation_accuracy_values = []

# For each epoch...
for epoch_i in range(0, 10):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0
    total_accuracy = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear gradients before each instance
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits

        # Accumulate the training loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Clip the norm of the gradients to prevent explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters
        optimizer.step()
        scheduler.step()

        # Track the accuracy
        preds = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_accuracy += accuracy_score(np.argmax(preds, axis=1), label_ids)

    # Calculate the average loss and accuracy over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    avg_train_accuracy = total_accuracy / len(train_dataloader)

    # Store the loss and accuracy values for plotting
    train_loss_values.append(avg_train_loss)
    train_accuracy_values.append(avg_train_accuracy)

    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Average training accuracy: {0:.4f}".format(avg_train_accuracy))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)

      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch

      # Telling the model not to compute or store gradients, saving memory and
      # speeding up validation
      with torch.no_grad():
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

        # Convert logits to PyTorch tensor if it's a NumPy array
        logits = torch.from_numpy(logits)

        # Ensure label type compatibility
        b_labels = b_labels.long()
        logits = logits.to(b_labels.device)

        # Calculate the evaluation loss for this batch and accumulate it
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
        eval_loss += loss.item()

    # Store the validation loss and accuracy values
    validation_loss_values.append(eval_loss / len(validation_dataloader))
    validation_accuracy_values.append(eval_accuracy / len(validation_dataloader))


    # Report the final accuracy, loss, and time taken for this validation run.
    print("  Validation Accuracy: {0:.4f}".format(eval_accuracy/len(validation_dataloader)))
    print("  Validation Loss: {0:.4f}".format(eval_loss/len(validation_dataloader)))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))



print("")
print("Training complete!")





======== Epoch 1 / 4 ========
Training...
  Average training loss: 0.5339
  Average training accuracy: 0.7215
  Training epoch took: 0:01:26

Running Validation...
  Validation Accuracy: 0.8625
  Validation Loss: 0.3274
  Validation took: 0:00:03

======== Epoch 2 / 4 ========
Training...
  Average training loss: 0.2811
  Average training accuracy: 0.8979
  Training epoch took: 0:01:25

Running Validation...
  Validation Accuracy: 0.9062
  Validation Loss: 0.3046
  Validation took: 0:00:03

======== Epoch 3 / 4 ========
Training...
  Average training loss: 0.1257
  Average training accuracy: 0.9604
  Training epoch took: 0:01:25

Running Validation...
  Validation Accuracy: 0.8812
  Validation Loss: 0.3480
  Validation took: 0:00:03

======== Epoch 4 / 4 ========
Training...
  Average training loss: 0.0259
  Average training accuracy: 0.9910
  Training epoch took: 0:01:25

Running Validation...
  Validation Accuracy: 0.9000
  Validation Loss: 0.4691
  Validation took: 0:00:03

=======